In [12]:
import pandas as pd
import numpy as np
import os

BASE_DIR = os.getcwd()
PREPROCESSED_DIR = os.path.join(BASE_DIR, "../data/preprocessed")
PROCESSED_DIR = os.path.join(BASE_DIR, "../data/processed")
VISUALS_DIRS = os.path.join(BASE_DIR, "..", "visuals")

In [13]:
def setup_directories():
    """Create necessary directories if they don't exist."""
    data_dir = PREPROCESSED_DIR
    output_dir = PROCESSED_DIR
    visuals_dirs = VISUALS_DIRS
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(visuals_dirs, exist_ok=True)
    return data_dir, output_dir

In [14]:
def load_datasets(data_dir):
    """Load preprocessed datasets from CSV files."""
    files = {
        "price": "preprocessed_cleaned_Day-ahead_prices_202301010000_202503050000_Hour.csv",
        "actual_consumption": "preprocessed_cleaned_Actual_consumption_202301010000_202503050000_Quarterhour.csv",
        "forecast_consumption": "preprocessed_cleaned_Forecasted_consumption_202301010000_202503050000_Quarterhour.csv",
        "actual_generation": "preprocessed_cleaned_Actual_generation_202301010000_202503050000_Quarterhour.csv",
        "forecast_generation": "preprocessed_cleaned_Forecasted_generation_Day-Ahead_202301010000_202503050000_Hour_Quarterhour.csv",
        "cross_border_flows": "preprocessed_cleaned_Cross-border_physical_flows_202301010000_202503050000_Quarterhour.csv",
        "scheduled_exchanges": "preprocessed_cleaned_Scheduled_commercial_exchanges_202301010000_202503050000_Quarterhour.csv",
    }
    
    dfs = {key: pd.read_csv(os.path.join(data_dir, file), delimiter=",", low_memory=False) for key, file in files.items()}
    return dfs

In [15]:
def clean_column_names(dfs):
    """Strip spaces and special characters from column names."""
    for df in dfs.values():
        df.columns = df.columns.str.strip().str.replace(r"[^\x00-\x7F]+", "", regex=True)

In [16]:
def preprocess_data(dfs):
    """Convert '-' to NaN and ensure numeric columns."""
    for df in dfs.values():
        df.replace("-", np.nan, inplace=True)
        df.infer_objects(copy=False)
        df[df.columns.difference(["Start date"])] = df[df.columns.difference(["Start date"])].apply(pd.to_numeric, errors="coerce")

In [17]:
def compute_average_price(df_price):
    """Calculate average price per MWh."""
    price_columns = [col for col in df_price.columns if "/MWh" in col]
    if not price_columns:
        raise KeyError("⚠ No columns with '/MWh' found for price data!")
    df_price["Average_Price_€/MWh"] = df_price[price_columns].mean(axis=1)

In [18]:
def merge_datasets(dfs):
    """Merge datasets on 'Start date' column."""
    df = dfs["price"]
    for key, data in dfs.items():
        if key != "price":
            df = df.merge(data, on="Start date", how="inner", suffixes=("", f"_{key}"))
    df = df.loc[:, ~df.columns.duplicated()]
    return df

In [19]:
def feature_engineering(df):
    """Add rolling averages, lag features, and volatility measures."""
    df["Rolling_Mean_24h"] = df["Average_Price_€/MWh"].rolling(window=24, min_periods=1).mean()
    df["Rolling_Mean_7d"] = df["Average_Price_€/MWh"].rolling(window=24*7, min_periods=1).mean()
    df["Price_Diff"] = df["Average_Price_€/MWh"].diff()
    df["Lag_1h"] = df["Average_Price_€/MWh"].shift(1)
    df["Lag_24h"] = df["Average_Price_€/MWh"].shift(24)
    df["Volatility_24h"] = df["Average_Price_€/MWh"].rolling(window=24, min_periods=1).std()
    df["Price_Change_1h"] = df["Average_Price_€/MWh"].pct_change() * 100
    df["Price_Change_24h"] = df["Average_Price_€/MWh"].pct_change(24) * 100
    return df

In [22]:
def save_processed_data(df, output_dir):
    """Save processed data in different time resolutions."""
    
    if "Start date" not in df.columns:
        raise KeyError("⚠ 'Start date' column is missing after processing!")

    # Ensure 'Start date' is a datetime type
    df["Start date"] = pd.to_datetime(df["Start date"], errors="coerce")
    
    # Drop rows where 'Start date' could not be converted
    df = df.dropna(subset=["Start date"])

    # Set index for resampling
    df.set_index("Start date", inplace=True)

    # Now resample and save
    df.resample("H").mean().to_csv(os.path.join(output_dir, "processed_hourly_data.csv"), sep=",")
    df.resample("D").mean().to_csv(os.path.join(output_dir, "processed_daily_data.csv"), sep=",")
    df.resample("W").mean().to_csv(os.path.join(output_dir, "processed_weekly_data.csv"), sep=",")

    print("✅ Processed data saved successfully!")


In [23]:
def main():
    """Main function to run data processing pipeline."""
    data_dir, output_dir = setup_directories()
    dfs = load_datasets(data_dir)
    clean_column_names(dfs)
    preprocess_data(dfs)
    compute_average_price(dfs["price"])
    df_merged = merge_datasets(dfs)
    df_final = feature_engineering(df_merged)
    save_processed_data(df_final, output_dir)

if __name__ == "__main__":
    main()

C:\Users\Elite\AppData\Local\Temp\ipykernel_19804\1655499764.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("-", np.nan, inplace=True)
C:\Users\Elite\AppData\Local\Temp\ipykernel_19804\759571360.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df.resample("H").mean().to_csv(os.path.join(output_dir, "processed_hourly_data.csv"), sep=",")


✅ Processed data saved successfully!
